# Tutorial: Predicción de Mora con PCA y RandomForest

## Introducción

Este notebook te guiará a través de la construcción de un modelo para predecir la 'Mora' (incumplimiento de pago) de los clientes. Exploraremos el uso del Análisis de Componentes Principales (PCA) para la reducción de dimensionalidad antes de entrenar un clasificador RandomForest.

**Puntos Clave del Tutorial:**
1.  Preparación de datos para la predicción.
2.  Un primer intento de modelado, donde identificaremos un problema común: la fuga de datos (data leakage).
3.  Corrección del problema de fuga de datos y un segundo intento de modelado.
4.  Aplicación de PCA para reducir la dimensionalidad del conjunto de características.
5.  Entrenamiento y evaluación de un RandomForestClassifier utilizando los componentes principales.
6.  Un método para estimar la importancia de las características originales después de aplicar PCA.

Este enfoque iterativo es común en el desarrollo de modelos de machine learning.

## 1. Configuración del Entorno y Carga de Datos

### 1.1 Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### 1.2 Descarga y Carga del DataFrame `df_analisis`

Usaremos `df_analisis.parquet`, que contiene datos detallados de facturas y pagos, previamente procesado.

In [ ]:
!wget -N https://github.com/srJboca/segmentacion/raw/refs/heads/main/archivos/df_analisis.parquet
df_analisis = pd.read_parquet('df_analisis.parquet')

## 2. Preparación Inicial de Datos para Predicción

Seleccionaremos las características relevantes y realizaremos las transformaciones necesarias.

In [ ]:
# Eliminar la columna 'mora' (minúsculas) si existe 'Mora' (mayúsculas) para evitar confusión
if 'mora' in df_analisis.columns and 'Mora' in df_analisis.columns:
    df_analisis = df_analisis.drop(columns=['mora'])
elif 'mora' in df_analisis.columns and 'Mora' not in df_analisis.columns:
    df_analisis.rename(columns={'mora': 'Mora'}, inplace=True)

# Selección inicial de características para el DataFrame de predicción
df_prediccion = df_analisis[[
    'Numero de factura', # Se eliminará más adelante, útil para inspección inicial
    'Consumo (m3)', 
    'Estrato', 
    'Precio por Consumo', 
    'Dias_Emision_PagoOportuno', 
    'Dias_Lectura_Emision', 
    'Dias_PagoOportuno_PagoReal', # Potencial fuente de data leakage
    'Mora' # Variable objetivo
]].copy()

print("--- df_prediccion (primeras filas) ---")
print(df_prediccion.head())

### 2.1 Preprocesamiento Básico
* Eliminar identificadores ('Numero de factura').
* Convertir 'Estrato' a numérico.
* Manejar valores faltantes (NaN) mediante imputación con la media.

In [ ]:
df_prediccion_ml = df_prediccion.drop('Numero de factura', axis=1).copy()

# Convertir 'Estrato' a numérico ordinal
if df_prediccion_ml['Estrato'].dtype == 'object' or isinstance(df_prediccion_ml['Estrato'].dtype, pd.CategoricalDtype):
    df_prediccion_ml['Estrato'] = df_prediccion_ml['Estrato'].str.replace('Estrato ', '', regex=False).astype(int)
else:
    df_prediccion_ml['Estrato'] = df_prediccion_ml['Estrato'].astype(int)

# Imputar NaNs con la media de cada columna (estrategia del notebook original)
for col in df_prediccion_ml.columns:
    if df_prediccion_ml[col].isnull().any():
        df_prediccion_ml[col].fillna(df_prediccion_ml[col].mean(), inplace=True)

print("--- df_prediccion_ml después de preprocesamiento básico (primeras filas) ---")
print(df_prediccion_ml.head())
print("\n--- Valores faltantes restantes ---")
print(df_prediccion_ml.isnull().sum())

## 3. Modelado (Intento 1): Con Posible Fuga de Datos

En este primer intento, usaremos todas las características preparadas, incluyendo `Dias_PagoOportuno_PagoReal`. Esta columna se deriva directamente de la fecha de pago real y la fecha de pago oportuno, y la variable 'Mora' también se define a partir de esta diferencia. Esto puede causar **fuga de datos (data leakage)**, donde el modelo aprende una relación demasiado directa y obtiene un rendimiento artificialmente alto.


### 3.1 Definición de X e y, Escalado y PCA

In [ ]:
X_leak = df_prediccion_ml.drop('Mora', axis=1)
y_leak = df_prediccion_ml['Mora']

# Escalado de características
scaler_leak = StandardScaler()
X_scaled_leak = scaler_leak.fit_transform(X_leak)

# Aplicación de PCA
# n_components=0.95 significa que PCA seleccionará el número de componentes 
# que explican el 95% de la varianza en los datos.
pca_leak = PCA(n_components=0.95)
X_pca_leak = pca_leak.fit_transform(X_scaled_leak)

print(f"Número de componentes seleccionados por PCA (con leakage): {pca_leak.n_components_}")
print(f"Varianza explicada por los componentes: {pca_leak.explained_variance_ratio_}")
print(f"Varianza explicada acumulada: {pca_leak.explained_variance_ratio_.sum():.4f}")

### 3.2 División de Datos, Entrenamiento y Evaluación (Intento 1)

In [ ]:
X_train_pca_leak, X_test_pca_leak, y_train_leak, y_test_leak = train_test_split(
    X_pca_leak, y_leak, test_size=0.2, random_state=42, stratify=y_leak
)

model_leak = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_leak.fit(X_train_pca_leak, y_train_leak)

y_pred_leak = model_leak.predict(X_test_pca_leak)

print("--- Evaluación del Modelo (Intento 1 - con posible leakage) ---")
print(f"Accuracy: {accuracy_score(y_test_leak, y_pred_leak):.4f}")
print("\nClassification Report:\n", classification_report(y_test_leak, y_pred_leak))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_leak, y_pred_leak))

**Observación del Intento 1:**
Los resultados (probablemente un accuracy perfecto o cercano al 100%) sugieren fuertemente la presencia de fuga de datos. La característica `Dias_PagoOportuno_PagoReal` contiene información que define directamente la variable `Mora`.

## 4. Modelado (Intento 2): Corrigiendo la Fuga de Datos

Ahora, eliminaremos la característica `Dias_PagoOportuno_PagoReal` de nuestro conjunto de datos `X` para evitar la fuga de datos y construir un modelo más realista y útil.

### 4.1 Preparación de Datos Sin Fuga, Escalado y PCA

In [ ]:
df_prediccion_ml_sin_leak = df_prediccion_ml.drop('Dias_PagoOportuno_PagoReal', axis=1)

X_sin_leak = df_prediccion_ml_sin_leak.drop('Mora', axis=1)
y_sin_leak = df_prediccion_ml_sin_leak['Mora']

# Escalado de características
scaler_sin_leak = StandardScaler()
X_scaled_sin_leak = scaler_sin_leak.fit_transform(X_sin_leak)

# Aplicación de PCA
pca_sin_leak = PCA(n_components=0.95) # Mantener 95% de la varianza
X_pca_sin_leak = pca_sin_leak.fit_transform(X_scaled_sin_leak)

print(f"Número de componentes seleccionados por PCA (sin leakage): {pca_sin_leak.n_components_}")
print(f"Varianza explicada por los componentes: {pca_sin_leak.explained_variance_ratio_}")
print(f"Varianza explicada acumulada: {pca_sin_leak.explained_variance_ratio_.sum():.4f}")

### 4.2 División de Datos, Entrenamiento y Evaluación (Intento 2)

In [ ]:
X_train_pca_sin_leak, X_test_pca_sin_leak, y_train_sin_leak, y_test_sin_leak = train_test_split(
    X_pca_sin_leak, y_sin_leak, test_size=0.2, random_state=42, stratify=y_sin_leak
)

model_sin_leak = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model_sin_leak.fit(X_train_pca_sin_leak, y_train_sin_leak)

y_pred_sin_leak = model_sin_leak.predict(X_test_pca_sin_leak)

print("--- Evaluación del Modelo (Intento 2 - sin leakage) ---")
print(f"Accuracy: {accuracy_score(y_test_sin_leak, y_pred_sin_leak):.4f}")
print("\nClassification Report:\n", classification_report(y_test_sin_leak, y_pred_sin_leak))

cm_sin_leak = confusion_matrix(y_test_sin_leak, y_pred_sin_leak)
print("\nConfusion Matrix:\n", cm_sin_leak)

# Visualización de la Matriz de Confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm_sin_leak, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Mora (0)', 'Mora (1)'], yticklabels=['No Mora (0)', 'Mora (1)'])
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión (Modelo sin Fuga de Datos)')
plt.show()

**Observación del Intento 2:**
Los resultados de este segundo modelo son más realistas y reflejan el verdadero poder predictivo de las características restantes después de aplicar PCA. Aunque el accuracy puede ser menor que en el intento con fuga de datos, este modelo es el que proporcionaría valor real en un escenario de producción.

## 5. Estimación de la Importancia de las Características Originales Post-PCA

El modelo RandomForest se entrenó con los componentes principales, no con las características originales. Sin embargo, podemos intentar estimar la importancia de las características originales analizando cómo contribuyen a los componentes principales más importantes.

**Método:**
1.  Obtener la importancia de cada componente principal del modelo RandomForest.
2.  Obtener los "loadings" de PCA (`pca.components_`), que indican cuánto contribuye cada característica original a cada componente principal (en valor absoluto).
3.  Multiplicar la importancia de cada componente por el valor absoluto de sus loadings y sumar estas contribuciones para cada característica original.

In [ ]:
# Importancia de los componentes principales del modelo
pca_component_importances = model_sin_leak.feature_importances_

# Loadings (contribución de características originales a cada componente)
# pca_sin_leak.components_ tiene forma (n_components, n_features_originales)
original_feature_loadings = np.abs(pca_sin_leak.components_)

# Ponderar los loadings por la importancia del componente
weighted_loadings = original_feature_loadings * pca_component_importances[:, np.newaxis]

# Sumar las contribuciones ponderadas para cada característica original
estimated_original_feature_importances = weighted_loadings.sum(axis=0)

# Normalizar para que sumen 1 (opcional, para comparación)
estimated_original_feature_importances_normalized = estimated_original_feature_importances / estimated_original_feature_importances.sum()

feature_names_sin_leak = X_sin_leak.columns
importance_df = pd.DataFrame({
    'Feature': feature_names_sin_leak,
    'Estimated_Importance': estimated_original_feature_importances_normalized
}).sort_values(by='Estimated_Importance', ascending=False)

print("--- Importancia Estimada de Características Originales Post-PCA ---")
print(importance_df)

plt.figure(figsize=(10, 7))
sns.barplot(x='Estimated_Importance', y='Feature', data=importance_df)
plt.title('Importancia Estimada de Características Originales (Post-PCA)')
plt.show()

**Interpretación de la Importancia Estimada:**
Este gráfico y tabla nos dan una idea de qué características originales fueron las más influyentes en el modelo final, incluso después de la transformación PCA. Las características con mayor importancia estimada son aquellas que tienen una fuerte presencia en los componentes principales que, a su vez, fueron importantes para el clasificador.

## 6. Conclusiones y Próximos Pasos

En este tutorial, hemos:
1.  Preparado datos para un modelo de predicción de 'Mora'.
2.  Demostrado la importancia de identificar y corregir la fuga de datos (`Dias_PagoOportuno_PagoReal`).
3.  Aplicado PCA para reducir la dimensionalidad, seleccionando componentes que explican el 95% de la varianza.
4.  Entrenado un RandomForestClassifier con los componentes principales.
5.  Evaluado el modelo y obtenido métricas de rendimiento más realistas.
6.  Estimado la importancia de las características originales después de PCA.

**Resultados Clave:**
* El modelo inicial con fuga de datos mostró un rendimiento perfecto, lo cual es una señal de alarma.
* El modelo corregido, sin la característica que causaba la fuga y utilizando PCA, arrojó un rendimiento (especificar accuracy y F1-score de la clase 'Mora'=1 según los resultados que se obtendrían) que refleja mejor la capacidad predictiva real.
* PCA redujo el número de características de (número original de `X_sin_leak.shape[1]`) a (número de `pca_sin_leak.n_components_`), simplificando potencialmente el modelo.
* El análisis de importancia de características post-PCA nos da una idea de qué factores originales son más relevantes.

**Próximos Pasos Sugeridos:**
* **Optimización de Hiperparámetros:** Tanto para RandomForest como para PCA (e.g., `n_components` podría ajustarse usando cross-validation o explorando diferentes umbrales de varianza explicada).
* **Comparación de Modelos:** Probar otros algoritmos de clasificación con y sin PCA.
* **Ingeniería de Características Adicional:** Crear nuevas características que podrían mejorar la predicción de la mora.
* **Análisis de Impacto de PCA:** Entrenar un modelo con el conjunto de características escaladas *sin* PCA y comparar su rendimiento (y tiempo de entrenamiento) con el modelo que usa PCA. Esto ayudaría a cuantificar el beneficio o costo de usar PCA en este problema específico.
* **Interpretabilidad del Modelo:** Si la interpretabilidad es crucial, explorar modelos más simples o técnicas de explicación de modelos (como SHAP) aplicadas a los componentes o, con más cuidado, a las características originales reconstruidas.